In [ ]:
# test the use of runlength encoding for rainfields rain_rate data 

In [52]:
import pymongo 
from pymongo import MongoClient
from netCDF4 import Dataset 
import collections
import bson

In [53]:
rf3_name="/home/awseed/data/RF3/prcp-crate/2021/20210129/310_20210129_003000.prcp-crate.nc" 
rf3 = Dataset(rf3_name)
stn_id = int(rf3.getncattr("station_id"))

In [54]:
# first check if the projection already exists in the database 
client = MongoClient()
db = client["test"]
collection = db["projection"] 
# get the number of documents for stn_id 
count_docs = collection.count_documents(filter={"stn_id": stn_id})
print(f"Found {count_docs} documents for this projection") 



Found 6 documents for this projection


In [55]:
proj = {}
if count_docs == 0 : 
     
    # get the projection as a dictionary 
    proj_attr = {}
    proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
    standard_parallel = rf3.variables['proj'].getncattr("standard_parallel")
    proj_attr['standard_parallel'] =  standard_parallel.tolist()
    proj_attr['longitude_of_central_meridian'] = rf3.variables['proj'].getncattr("longitude_of_central_meridian")
    proj_attr['latitude_of_projection_origin'] = rf3.variables['proj'].getncattr("latitude_of_projection_origin")
    proj_attr['false_easting'] = rf3.variables['proj'].getncattr("false_easting") 
    proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
    towgs84 = rf3.variables['proj'].getncattr("towgs84")
    proj_attr['towgs84'] = towgs84.tolist()
    
    x_attr = rf3.variables['x'].__dict__ 
    x = rf3['x'][:].tolist()
    x_attr['x'] = x 
    
    y_attr = rf3.variables['y'].__dict__ 
    y = rf3['y'][:].tolist()
    y_attr['y'] = y 
    
    proj['stn_id'] = int(stn_id)
    proj['proj'] = proj_attr 
    proj['y'] = y_attr
    proj['x'] = x_attr

    proj_id = collection.insert_one(proj).inserted_id
else:
    # get the projection from the database 
    proj = collection.find_one({"stn_id": stn_id}) 
    

{'_id': ObjectId('610cc5228db653a300998957'), 'stn_id': 310, 'proj': {'grid_mapping_name': 'albers_conical_equal_area', 'standard_parallel': [-18.0, -36.0], 'longitude_of_central_meridian': 132.0, 'latitude_of_projection_origin': 0.0, 'false_easting': 0.0, 'towgs84': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, 'y': {'standard_name': 'projection_y_coordinate', 'units': 'km', 'bounds': 'y_bounds', 'y': [-1001.0, -1003.0, -1005.0, -1007.0, -1009.0, -1011.0, -1013.0, -1015.0, -1017.0, -1019.0, -1021.0, -1023.0, -1025.0, -1027.0, -1029.0, -1031.0, -1033.0, -1035.0, -1037.0, -1039.0, -1041.0, -1043.0, -1045.0, -1047.0, -1049.0, -1051.0, -1053.0, -1055.0, -1057.0, -1059.0, -1061.0, -1063.0, -1065.0, -1067.0, -1069.0, -1071.0, -1073.0, -1075.0, -1077.0, -1079.0, -1081.0, -1083.0, -1085.0, -1087.0, -1089.0, -1091.0, -1093.0, -1095.0, -1097.0, -1099.0, -1101.0, -1103.0, -1105.0, -1107.0, -1109.0, -1111.0, -1113.0, -1115.0, -1117.0, -1119.0, -1121.0, -1123.0, -1125.0, -1127.0, -1129.0, -1131.0, -1133.0